# Run from jupyter

This page give an overview of the details of running a jupyter kernel from the jupyter notebook. It doesn't have much practical sence but extremely usefull for purposes of this project.

In [1]:
import multiprocessing
from ipykernel.kernelbase import Kernel
from ipykernel.kernelapp import IPKernelApp
from jupyter_client.blocking import BlockingKernelClient

## Issue

In [2]:
Kernel.instance()

In [3]:
try:
    IPKernelApp.launch_instance()
except Exception as e:
    print(e)

init_sockets cannot be called twice!


## Solution

In [4]:
%%writefile /tmp/run_kernel.py
import os
import sys
from ipykernel.kernelapp import IPKernelApp

def process_target():
    print(os.getcwd())
    IPKernelApp.launch_instance(["-f", "/tmp/example_connection.json"])

Overwriting /tmp/run_kernel.py


In [6]:
import sys 
sys.path.append("/tmp")
from run_kernel import process_target

context = multiprocessing.get_context("spawn")
p = context.Process(target=process_target)
p.start()

/home/fedor/Documents/python/packages/jupyter/kernel
NOTE: When using the `ipython kernel` entry point, Ctrl-C will not work.

To exit, you will have to explicitly quit this process, by either sending
"quit" from a client, or using Ctrl-\ in UNIX-like environments.

To read more about this, see https://github.com/ipython/ipython/issues/2049


To connect another client to this kernel, use:
    --existing /tmp/example_connection.json


In [7]:
client = BlockingKernelClient()
client.load_connection_file("/tmp/example_connection.json")
client.start_channels()
client.is_alive()

True